# Installing required packages

In [ ]:
!pip install nemo_toolkit[all]
!pip install pydub
!pip install jsonlines
!pip install pyaspeller

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nemo
import nemo.collections.asr as nemo_asr
import pytorch_lightning as pl
import pandas as pd
import jsonlines
from sklearn.model_selection import train_test_split
from jiwer import wer, cer
import gc
import torch
from pyaspeller import YandexSpeller
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

[NeMo W 2023-05-26 14:02:39 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-05-26 14:02:43 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


Mounted at /content/drive


# Load the QuatrzNet model which was trained on Zapadnodvisk and Opochka data

In [ ]:
asr_model_quartznet = nemo_asr.models.EncDecCTCModel.load_from_checkpoint('/content/drive/MyDrive/epoch=19-step=3960.ckpt')

[NeMo W 2023-05-26 14:03:20 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/train.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 8
    trim_silence: false
    max_duration: 37.02
    min_duration: 0.01
    num_workers: 0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2023-05-26 14:03:20 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a vali

[NeMo I 2023-05-26 14:03:20 features:287] PADDING: 16


# Download the manifest

In [ ]:
manifest = []
with jsonlines.open('/content/drive/MyDrive/manifest (1).jsonl') as f:
    for obj in f:
        manifest.append(obj)

train, test = train_test_split(manifest, test_size=0.3, random_state=22)

with jsonlines.open('train.jsonl', 'w') as writer:
    writer.write_all(train)
with jsonlines.open('test.jsonl', 'w') as writer:
    writer.write_all(test)

In [ ]:
asr_model_quartznet = asr_model_quartznet.cuda()

# Prepare function for evaluating

In [ ]:
def get_metrics(transcriptions):
    wers = []
    cers = []

    for i, transcription in enumerate(transcriptions):
        if test[i]['text'] != '' and test[i]['text'] != ' ':
          w = wer(test[i]['text'], transcription)
          wers.append(w)
          c = cer(test[i]['text'], transcription)
          cers.append(c)

    print('Mean WER: ', sum(wers)/len(wers))
    print('Mean CER: ', sum(cers)/len(cers))

In [ ]:
gc.collect()

if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

if accelerator == 'gpu':
  torch.cuda.empty_cache()

# QuarztNet: transcribe

In [ ]:
files = [file['audio_filepath'] for file in test]
transcriptions = asr_model_quartznet.transcribe(paths2audio_files=files)
get_metrics(transcriptions)

Transcribing:   0%|          | 0/595 [00:00<?, ?it/s]

Mean WER:  0.6159112562722011
Mean CER:  0.3112444169146459


In [ ]:
for i, item in enumerate(test):
    test[i]['transcription'] = transcriptions[i]
df_quartz = pd.DataFrame.from_records(test)
df_quartz.to_excel("output_quartz_shetnevo_after_opochka.xlsx") 

# Load the Conformer-Transducer model which was trained on Zapadnodvisk and Opocka data

In [ ]:
asr_model_conformer = nemo_asr.models.EncDecRNNTBPEModel.restore_from('/content/drive/MyDrive/finetuned_conformer_opochka.nemo')

[NeMo I 2023-05-26 14:41:15 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-05-26 14:41:16 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/train.jsonl
    sample_rate: 16000
    batch_size: 2
    shuffle: true
    num_workers: 0
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 25.375079365079365
    min_duration: 0.01
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    bucketing_weights: ''
    
[NeMo W 2023-05-26 14:41:16 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /content/test.jsonl
    sample_rat

[NeMo I 2023-05-26 14:41:16 features:287] PADDING: 0


[NeMo W 2023-05-26 14:41:18 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    
[NeMo W 2023-05-26 14:41:18 rnnt:1211] `preserve_memory` was set for the Joint Model. Please be aware this will severely impact the forward-backward step time. It also might not solve OOM issues if the GPU simply does not have enough memory to compute the joint.


[NeMo I 2023-05-26 14:41:18 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-05-26 14:41:19 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /content/drive/MyDrive/finetuned_conformer_opochka.nemo.


# Conformer-Transducer: transcribe

In [ ]:
asr_model_conformer = asr_model_conformer.cuda()
files = [file['audio_filepath'] for file in test]
transcriptions = asr_model_conformer.transcribe(paths2audio_files=files)
get_metrics(transcriptions[0])

Mean WER:  0.47741720477496175
Mean CER:  0.2983775604347399


In [ ]:
for i, item in enumerate(test):
    test[i]['transcription'] = transcriptions[0][i]
df_conformer = pd.DataFrame.from_records(test)
df_conformer.to_excel("output_conformer_shetnevo_after_opochka.xlsx") 

# Use spellchecker for the received transcriptions

In [ ]:
speller = YandexSpeller()
texts = list(df_quartz['text'])
transcriptions = list(df_quartz['transcription'])
transcrtiptions_spelled = []
for t in tqdm(transcriptions):
    transcrtiptions_spelled.append(speller.spelled(t))
wers = []
cers = []


for i, transcription in enumerate(transcrtiptions_spelled):
    if texts[i] != '' and texts[i] != ' ':
        w = wer(texts[i], transcription)
        wers.append(w)
        c = cer(texts[i], transcription)
        cers.append(c)

print('Mean WER: ', sum(wers)/len(wers))
print('Mean CER: ', sum(cers)/len(cers))

100%|██████████| 2377/2377 [50:31<00:00,  1.28s/it]

Mean WER:  0.5635382316252234
Mean CER:  0.3131632254550811
